In [1]:
#--------------------------------------------------------------------------------------------------------------------------------------
# LIBRARIES
#--------------------------------------------------------------------------------------------------------------------------------------

from ipyleaflet import Map, basemaps, Polyline, Marker, CircleMarker, LayerGroup
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import HTML, Dropdown, IntRangeSlider, FloatRangeSlider, Output, Text, Image, VBox, HBox, Label, IntProgress
from IPython.display import display
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from functools import partial
import numpy as np
import scipy.io as sio
from scipy import interpolate as interp
import time, datetime
from math import comb, exp

#--------------------------------------------------------------------------------------------------------------------------------------
# HEADER
#--------------------------------------------------------------------------------------------------------------------------------------
header = HTML("<center><h1>Landslide-Tsurrogate Model v1.0 - Mayotte (France)</h1></center>", layout=Layout(height='auto',width='99%'))

#--------------------------------------------------------------------------------------------------------------------------------------
# lEFT PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#--------------
## instructions
#--------------

instruction_txt = Text(value='Select a Submarine Landslide Zone', 
                       placeholder='', 
                       description='', 
                       style=dict(text_color='blue'), 
                       disabled=True,
                       layout=Layout(width='99%')
                      ) 
#-----
## map
#-----

m=Map(basemap=basemaps.Esri.WorldImagery, center=[-12.85, 45.2], zoom=10)
m.layout.width='auto'
m.layout.height='auto'

#-----------------
## landslide zones
#-----------------

lon_piton = sio.loadmat('data/zone.mat')['lon_piton']
lat_piton = sio.loadmat('data/zone.mat')['lat_piton']
dist_piton = sio.loadmat('data/zone.mat')['dist_piton']
coords = list(zip(lat_piton.flatten(), lon_piton.flatten()))
line_piton = Polyline(
    locations=coords,
    color="white",
    weight=3,
    properties={"name": "Piton"}
)
m.add_layer(line_piton)
lon_piton_south = sio.loadmat('data/zone.mat')['lon_piton_south']
lat_piton_south = sio.loadmat('data/zone.mat')['lat_piton_south']
dist_piton_south = sio.loadmat('data/zone.mat')['dist_piton_south']
coords = list(zip(lat_piton_south.flatten(), lon_piton_south.flatten()))
line_piton_south = Polyline(
    locations=coords,
    color="white",
    weight=3,
    properties={"name": "Piton South"}
)
m.add_layer(line_piton_south)
lon_piton_offshore = sio.loadmat('data/zone.mat')['lon_piton_offshore']
lat_piton_offshore = sio.loadmat('data/zone.mat')['lat_piton_offshore']
dist_piton_offshore = sio.loadmat('data/zone.mat')['dist_piton_offshore']
coords = list(zip(lat_piton_offshore.flatten(), lon_piton_offshore.flatten()))
line_piton_offshore = Polyline(
    locations=coords,
    color="white",
    weight=3,
    properties={"name": "Piton Offshore"}
)
m.add_layer(line_piton_offshore)
lon_mayotte_south = sio.loadmat('data/zone.mat')['lon_mayotte_south']
lat_mayotte_south = sio.loadmat('data/zone.mat')['lat_mayotte_south']
dist_mayotte_south = sio.loadmat('data/zone.mat')['dist_mayotte_south']
coords = list(zip(lat_mayotte_south.flatten(), lon_mayotte_south.flatten()))
line_mayotte_south = Polyline(
    locations=coords,
    color="white",
    weight=3,
    properties={"name": "Mayotte South"}
)
m.add_layer(line_mayotte_south)
lon_mayotte_offshore = sio.loadmat('data/zone.mat')['lon_mayotte_offshore']
lat_mayotte_offshore = sio.loadmat('data/zone.mat')['lat_mayotte_offshore']
dist_mayotte_offshore = sio.loadmat('data/zone.mat')['dist_mayotte_offshore']
coords = list(zip(lat_mayotte_offshore.flatten(), lon_mayotte_offshore.flatten()))
line_mayotte_offshore = Polyline(
    locations=coords,
    color="white",
    weight=3,
    properties={"name": "Mayotte Offshore"}
)
m.add_layer(line_mayotte_offshore)

#--------------------------------------------------------------------------------------------------------------------------------------
# MIDDLE PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#------------------------
## CLEAR selector: RESET
#------------------------

CLEAR_selector = Button(description='CLEAR', 
                     disabled=False,
                     button_style='danger',
                     tooltip='Reset all results',
                     icon=''
                    )
CLEAR_selector_out = Output()
def update_CLEAR(b):
    with CLEAR_selector_out:        
        CLEAR_selector_out.clear_output()
        m.center=[-12.85, 45.2]
        m.zoom=10        
        instruction_txt.value = 'Select a Submarine Landslide Zone'
        zone_selector.value=''
        zone_selector.disabled=False
        area_selector.value=''
        area_selector.disabled=True
        distance_selector.value=[100, 500] 
        distance_selector.min=0 
        distance_selector.max=919.0
        distance_selector.disabled=True
        volume_selector.value=[10, 50] 
        volume_selector.min=1 
        volume_selector.max=200.0 
        volume_selector.disabled=True
        friction_selector.value=[7, 9] 
        friction_selector.min=4 
        friction_selector.max=12 
        friction_selector.disabled=True
        param_box.layout.display = 'none'
        OK_selector.disabled=True
        OK_box.layout.display = 'none'
        OK_selector_out.clear_output()
        progress.value = 0        
        fig_zeta_box.layout.display = 'none'
        fig_velo_box.layout.display = 'none'
        fig_time_box.layout.display = 'none'
        stats_box.layout.display = 'none'
        stat_ele1_txt.value=''
        stat_ele2_txt.value=''
        stat_ele3_txt.value=''
        stat_ele4_txt.value=''
        stat_ele5_txt.value=''
        stat_ele6_txt.value=''
        stat_time1_txt.value=''
        stat_time2_txt.value=''
        stat_time3_txt.value=''
        stat_time4_txt.value=''
        stat_time5_txt.value=''
        stat_time6_txt.value=''
        loc_box.layout.display = 'none'
        loc_number_txt.value = ''
        loc_lon_txt.value = ''
        loc_lat_txt.value = ''
        if layer_group is not None:
            m.remove_layer(layer_group)
        try:
            line_display
        except NameError:
            line_display = None
        else:
            if line_display in m.layers:
                m.remove_layer(line_display)
        for name, line in zone_lines.items():
            m.remove_layer(line)
            line.color = "white"
            m.add_layer(line)            
CLEAR_selector.on_click(update_CLEAR)

#-------------------------
## landslide zone selector
#-------------------------

zone_selector = Dropdown( options = ('','Piton','Piton Offshore','Piton South','Mayotte South','Mayotte Offshore'), 
                            layout=Layout(width='auto'))
zone_out = Output()
zone_lines = {
    "Piton": line_piton,
    "Piton South": line_piton_south,
    "Piton Offshore": line_piton_offshore,
    "Mayotte South": line_mayotte_south,
    "Mayotte Offshore": line_mayotte_offshore
}
zone_dist = {
    "Piton": dist_piton.flatten(),
    "Piton South": dist_piton_south.flatten(),
    "Piton Offshore": dist_piton_offshore.flatten(),
    "Mayotte South": dist_mayotte_south.flatten(),
    "Mayotte Offshore": dist_mayotte_offshore.flatten()
}
zone_max_dist = {
    "Piton": 919.0,
    "Piton South": 5045.0,
    "Piton Offshore": 1190.0,
    "Mayotte South": 6071.0,
    "Mayotte Offshore": 3568.0
}
zone_max_vol = {
    "Piton": 200.0,
    "Piton South": 50.0,
    "Piton Offshore": 100.0,
    "Mayotte South": 50.0,
    "Mayotte Offshore": 100.0
}
zone_file = {
    "Piton": "Piton_DGP_dim_3_order_6_coeff.mat",
    "Piton South": "Piton_South_DGP_dim_3_order_6_coeff.mat",
    "Piton Offshore": "Piton_Offshore_DGP_dim_3_order_6_coeff.mat",
    "Mayotte South": "Mayotte_South_DGP_dim_3_order_6_coeff.mat",
    "Mayotte Offshore": "Mayotte_Offshore_DGP_dim_3_order_6_coeff.mat"
}
interp_funcs = {}
surrogate_name = []
def update_zone(change):
    global surrogate_name
    with zone_out:
        zone_out.clear_output()
        selected = change['new']
        for name, line in zone_lines.items():
            m.remove_layer(line)  # remove old
            if name == selected:
                line.color = "red"
            else:
                line.color = "white"
            m.add_layer(line) 
        # Interpolation rule from distance to coordinates & Zoom map to selected zone
        if selected in zone_lines:
            dist = zone_dist[selected]
            line = zone_lines[selected]
            lats, lons = zip(*line.locations)
            m.center = [sum(lats)/len(lats), sum(lons)/len(lons)]
            m.zoom = 13
            interp_funcs['lon'] = interp.interp1d(dist, lons, fill_value="extrapolate")
            interp_funcs['lat'] = interp.interp1d(dist, lats, fill_value="extrapolate")
            instruction_txt.value = 'Select a Coastal Area or Push CLEAR'
            area_selector.disabled=False
            area_selector.value=''
        else:
            m.center=[-12.85, 45.2]
            m.zoom=10
            instruction_txt.value = 'Select a Submarine Landslide Zone'
            area_selector.disabled=True
            area_selector.value=''
        if selected in zone_max_dist:
            distance_selector.max=zone_max_dist[selected]
        if selected in zone_max_vol:
            volume_selector.max=zone_max_vol[selected]
        if selected in zone_file:
            surrogate_name = zone_file[selected]
zone_selector.observe(update_zone, names='value')

#-----------------------
## coastal area selector
#-----------------------

area_selector = Dropdown( options = ('','Petite Terre','Grande Terre'), disabled=True, 
                            layout=Layout(width='auto'))
area_out = Output()
coeff = []
n_locations = 0
def update_area(change):
    global coeff, lon_coast, lat_coast
    with area_out:
        area_out.clear_output()
        zone_selector.disabled=True        
        selected = change['new']
        if selected == "Petite Terre":
            instruction_txt.value = 'Select the Landslide Parameters (distance,volume, friction) or Push CLEAR'
            lon_coast = sio.loadmat('data/location.mat')['lon_petite']
            lat_coast = sio.loadmat('data/location.mat')['lat_petite']
            distance_selector.disabled=False
            volume_selector.disabled=False
            friction_selector.disabled=False
            param_box.layout.display = 'flex'
            OK_selector.disabled=False
            OK_box.layout.display = 'flex'
            coeff = load_coeff('data/' + surrogate_name, 'coeff_hat_petite')
            area_selector.disabled=True
        else:
            if selected == "Grande Terre":
                instruction_txt.value = 'Select the Landslide Parameters (distance,volume, friction) or Push CLEAR'
                lon_coast = sio.loadmat('data/location.mat')['lon_grande']
                lat_coast = sio.loadmat('data/location.mat')['lat_grande']
                distance_selector.disabled=False
                volume_selector.disabled=False
                friction_selector.disabled=False
                param_box.layout.display = 'flex'
                OK_selector.disabled=False
                OK_box.layout.display = 'flex'
                coeff = load_coeff('data/' + surrogate_name, 'coeff_hat_grande')
                area_selector.disabled=True
            else:
                instruction_txt.value = 'Select a Coastal Area'
area_selector.observe(update_area, names='value')

def load_coeff(mat_file, var_name):
    mat = sio.loadmat(mat_file, squeeze_me=True, struct_as_record=False)
    arr = mat[var_name]  
    arr = np.atleast_1d(arr)
    coeff_list = []
    for elem in arr:
        coeff_dict = {name: getattr(elem, name) for name in elem._fieldnames}
        coeff_list.append(coeff_dict)
    return coeff_list   
    
#-------------------
## distance selector
#-------------------

distance_selector = IntRangeSlider(value= [100, 500], 
                                   min=0, 
                                   max=919.0, 
                                   step=1, 
                                   disabled=True, 
                                   orientation='horizontal', 
                                   readout=True, 
                                   readout_format='.1f',
                                   layout=Layout(width='auto'))
distance_out = Output()
line_display = None
def update_distance(change):
    global line_display
    with distance_out:
        distance_out.clear_output(wait=True)
        selected = change['new']
        dist_display = np.arange(selected[0], selected[1] + 10, 10)
        lon_display = interp_funcs['lon'](dist_display)
        lat_display = interp_funcs['lat'](dist_display)
        coords_display = list(zip(lat_display, lon_display))
        if line_display is not None:
            m.remove_layer(line_display)
        line_display = Polyline(
            locations=coords_display,
            color="green",
            weight=3,
        )
        m.add_layer(line_display)
        instruction_txt.value = 'The selected portion of the zone is displayed in green'
distance_selector.observe(update_distance, names='value')

#-------------------
## volume selector
#------------------- 

volume_selector = IntRangeSlider(value= [10, 50], 
                                   min=1, 
                                   max=200.0, 
                                   step=1, 
                                   disabled=True, 
                                   continuous_update=True, 
                                   orientation='horizontal', 
                                   readout=True, 
                                   readout_format='.1f',
                                   layout=Layout(width='auto'))

#-------------------
## friction selector
#-------------------

friction_selector = FloatRangeSlider(value= [7, 9], 
                                   min=4, 
                                   max=12, 
                                   step=0.5, 
                                   disabled=True, 
                                   continuous_update=True, 
                                   orientation='horizontal', 
                                   readout=True, 
                                   readout_format='.1f',
                                   layout=Layout(width='auto'))


param_box = VBox([Label("Distance [m]:"), distance_selector, distance_out,
                  Label("Volume [Mm3]:"), volume_selector,
                  Label("Angle of friction [°]:"), friction_selector,])
param_box.layout.display = 'none'

#------------------------
## OK selector: model run
#------------------------

OK_selector = Button(description='OK', 
                     disabled=True,
                     button_style='success',
                     tooltip='Run the Model',
                     icon=''
                    )
OK_selector_out = Output()
layer_group=None
def update_OK(b):
    with OK_selector_out:
        global PTHA_zeta, PTHA_velo, PTHA_time, layers, layer_group
        OK_selector_out.clear_output(wait=True)        
        distance_selector.disabled=True
        volume_selector.disabled=True
        friction_selector.disabled=True
        OK_selector.disabled=True
        progress.value = 0
        a0 = np.array([distance_selector.value[0],volume_selector.value[0],friction_selector.value[0]])
        b0 = np.array([distance_selector.value[1],volume_selector.value[1],friction_selector.value[1]])
        aa = np.array([distance_selector.min,volume_selector.min,friction_selector.min])
        bb = np.array([distance_selector.max,volume_selector.max,friction_selector.max])
        nw0 = 1000
        maxdeg0 = 5
        start = time.time()
        PTHA_zeta, PTHA_velo, PTHA_time = surrogate_model_gauss_patterson_PTHA(maxdeg0, aa, bb, coeff, nw0, a0, b0, progress=progress)
        end = time.time()
        length = end - start
        print(str(nw0) + " model runs produced in " + str(length) + " s")
        # Initialize plots with location 0
        n_locs = coeff[0]['zeta'].shape[0]
        m.center = [sum(lat_coast.flatten())/len(lat_coast.flatten()), sum(lon_coast.flatten())/len(lon_coast.flatten())]
        m.zoom = 13
        layers = []
        for i in range(n_locs):
            lat = float(np.ravel(lat_coast[i])[0])
            lon = float(np.ravel(lon_coast[i])[0])
            markers = CircleMarker(location=(lat, lon), radius=8, color="black", fill_color="white", fill_opacity=0.8, draggable=False)
            markers.on_click(partial(on_marker_click, id=i))
            layers.append(markers)
        layer_group=LayerGroup(layers=layers)
        m.add_layer(layer_group)
        instruction_txt.value = "Click on the map to select locations or Push CLEAR"
        fig_zeta_box.layout.display = 'flex'
        fig_velo_box.layout.display = 'flex'
        fig_time_box.layout.display = 'flex'
        stats_box.layout.display = 'flex'
        val_zeta = PTHA_zeta[0,:]
        val_time = PTHA_time[0,:] / 60.0
        create_plot_widget(fig_zeta, val_zeta, 0, "Maximum Tsunami Elevation [m]", color="blue")
        create_plot_widget(fig_velo, PTHA_velo[0,:], 0,"Maximum Tsunami Speed [m/s]", color="green")
        create_plot_widget(fig_time, val_time, 1, "Tsunami Time of Arrival [mm:ss]", color="purple")
        loc_box.layout.display = 'flex'
        loc_number_txt.value = str(1)
        loc_lon_txt.value = str(lon_coast[0])
        loc_lat_txt.value = str(lat_coast[0])
        tot_zeta = len(val_zeta)
        zeta1 = np.sum(val_zeta <= 0.5) * 100 /tot_zeta
        zeta2 = np.sum((val_zeta > 0.5) & (val_zeta <= 1.0)) * 100 /tot_zeta
        zeta3 = np.sum((val_zeta > 1.0) & (val_zeta <= 3.0)) * 100 /tot_zeta
        zeta4 = np.sum((val_zeta > 3.0) & (val_zeta <= 5.0)) * 100 /tot_zeta
        zeta5 = np.sum((val_zeta > 5.0) & (val_zeta <= 10.0)) * 100 /tot_zeta
        zeta6 = np.sum(val_zeta > 10.0) * 100 /tot_zeta
        stat_ele1_txt.value=str(zeta1)
        stat_ele2_txt.value=str(zeta2)
        stat_ele3_txt.value=str(zeta3)
        stat_ele4_txt.value=str(zeta4)
        stat_ele5_txt.value=str(zeta5)
        stat_ele6_txt.value=str(zeta6)
        tot_time = len(val_time)
        time1 = np.sum(val_time <= 1.0) * 100 /tot_time
        time2 = np.sum((val_time > 1.0) & (val_time <= 2.0)) * 100 /tot_time
        time3 = np.sum((val_time > 2.0) & (val_time <= 3.0)) * 100 /tot_time
        time4 = np.sum((val_time > 3.0) & (val_time <= 5.0)) * 100 /tot_time
        time5 = np.sum((val_time > 5.0) & (val_time <= 10.0)) * 100 /tot_time
        time6 = np.sum(val_time > 10.0) * 100 /tot_zeta
        stat_time1_txt.value=str(time1)
        stat_time2_txt.value=str(time2)
        stat_time3_txt.value=str(time3)
        stat_time4_txt.value=str(time4)
        stat_time5_txt.value=str(time5)
        stat_time6_txt.value=str(time6)        
OK_selector.on_click(update_OK)
progress = IntProgress(
    value=0, 
    min=0, 
    max=100,
    description='',
    style={'bar_color': 'maroon'},
)
OK_box = HBox([OK_selector, progress, OK_selector_out])
OK_box.layout.display = 'none'
def surrogate_model_gauss_patterson_PTHA(maxdeg, aa, bb, coeff_hat, nw, ar, br, progress=None):      
    nmodes = len(ar)
    nx = len(coeff_hat[0]['zeta'])
    # Build Legendre polynomials up to maxdeg
    Le = [None] * (maxdeg + 1)
    Le[0] = np.array([1.0])         # L_0 = 1
    Le[1] = np.array([1.0, 0.0])    # L_1 = x
    for n in range(2, maxdeg + 1):
        Le[n] = ((2 * n - 1) / n) * np.concatenate((Le[n - 1], [0])) - ((n - 1) / n) * np.concatenate(([0, 0], Le[n - 2]))  
    # Generate random input  
    np.random.seed(0)
    seed_01 = np.random.rand(nmodes, nw)    
    # Rescale to intervals
    Zw = np.tile(ar, (nw,1)).T + np.tile((br - ar), (nw,1)).T * seed_01
    # Preallocate output
    zeta_temp = np.zeros((nx, nw))
    velo_temp = np.zeros((nx, nw))
    time_temp = np.zeros((nx, nw))
    # Total iterations for progress
    total_steps = (maxdeg + 1 - max(0, maxdeg - nmodes + 1))
    current_step = 0
    for alpha_norm1 in range(max(0, maxdeg - nmodes + 1), maxdeg + 1):
        # Smolyak coefficient
        C_alpha = ((-1)**(maxdeg - alpha_norm1) * comb(nmodes - 1, maxdeg - alpha_norm1))            
        # Retrieve coefficients
        alpha    = np.array(coeff_hat[alpha_norm1]['alpha']) 
        zeta_hat = np.array(coeff_hat[alpha_norm1]['zeta'])   
        velo_hat = np.array(coeff_hat[alpha_norm1]['velo'])
        time_hat = np.array(coeff_hat[alpha_norm1]['time'])
        nww = alpha.shape[0]            
        for l in range(nww):
            multH = np.ones(nw)
            for n in range(nmodes):
                x_scaled = (2 * Zw[n, :] - aa[n] - bb[n]) / (bb[n] - aa[n])
                multH *= np.polyval(Le[alpha[l, n]], x_scaled)
            for i in range(nx):
                zeta_temp[i, :] += C_alpha * zeta_hat[i, l] * multH
                velo_temp[i, :] += C_alpha * velo_hat[i, l] * multH
                time_temp[i, :] += C_alpha * time_hat[i, l] * multH
        # Update progress bar at each alpha_norm1 step
        current_step += 1
        if progress:
            progress.value = int(current_step / total_steps * progress.max)
    # Undo log transform
    zeta = np.exp(zeta_temp)
    velo = np.exp(velo_temp)
    time = np.exp(time_temp)
    return zeta, velo, time
# Create markers and add click events
def on_marker_click(event=None, id=None, **kwargs):
    loc_index = id
    loc_num = loc_index+1
    loc_number_txt.value = str(loc_num)
    loc_lon_txt.value = str(lon_coast[loc_index])
    loc_lat_txt.value = str(lat_coast[loc_index])  
    for i, marker in enumerate(layers):
        if i == id:
            marker.fill_color = "red"
        else:
            marker.fill_color = "white"
    val_zeta = PTHA_zeta[loc_index,:]
    val_time = PTHA_time[loc_index,:] / 60.0
    create_plot_widget(fig_zeta, val_zeta, 0, "Maximum Tsunami Elevation [m]", color="blue")
    create_plot_widget(fig_velo, PTHA_velo[loc_index,:], 0, "Maximum Tsunami Speed [m/s]", color="green")
    create_plot_widget(fig_time, val_time, 1, "Tsunami Time of Arrival [mm:ss]", color="purple")
    tot_zeta = len(val_zeta)
    zeta1 = np.sum(val_zeta <= 0.5) * 100 /tot_zeta
    zeta2 = np.sum((val_zeta > 0.5) & (val_zeta <= 1.0)) * 100 /tot_zeta
    zeta3 = np.sum((val_zeta > 1.0) & (val_zeta <= 3.0)) * 100 /tot_zeta
    zeta4 = np.sum((val_zeta > 3.0) & (val_zeta <= 5.0)) * 100 /tot_zeta
    zeta5 = np.sum((val_zeta > 5.0) & (val_zeta <= 10.0)) * 100 /tot_zeta
    zeta6 = np.sum(val_zeta > 10.0) * 100 /tot_zeta
    stat_ele1_txt.value=str(zeta1)
    stat_ele2_txt.value=str(zeta2)
    stat_ele3_txt.value=str(zeta3)
    stat_ele4_txt.value=str(zeta4)
    stat_ele5_txt.value=str(zeta5)
    stat_ele6_txt.value=str(zeta6)
    tot_time = len(val_time)
    time1 = np.sum(val_time <= 1.0) * 100 /tot_time
    time2 = np.sum((val_time > 1.0) & (val_time <= 2.0)) * 100 /tot_time
    time3 = np.sum((val_time > 2.0) & (val_time <= 3.0)) * 100 /tot_time
    time4 = np.sum((val_time > 3.0) & (val_time <= 5.0)) * 100 /tot_time
    time5 = np.sum((val_time > 5.0) & (val_time <= 10.0)) * 100 /tot_time
    time6 = np.sum(val_time > 10.0) * 100 /tot_zeta
    stat_time1_txt.value=str(time1)
    stat_time2_txt.value=str(time2)
    stat_time3_txt.value=str(time3)
    stat_time4_txt.value=str(time4)
    stat_time5_txt.value=str(time5)
    stat_time6_txt.value=str(time6)

#--------------------------------------------------------------------------------------------------------------------------------------
# RIGHT PANEL
#--------------------------------------------------------------------------------------------------------------------------------------

#-------------------------
## logo
#-------------------------

img_widget = Image(value=open("data/splash.png", "rb").read(), format='png', layout=Layout(width='60%'))

#-------------------------
## location information
#-------------------------

loc_header_txt =  HTML("<center><h3>Selected Location</h3></center>", layout=Layout(height='auto',width='99%'))
loc_number_txt = Text(value='', placeholder='',description='Number [#]',disabled=True)
loc_lon_txt = Text(value='', placeholder='',description='Long. [°E]',disabled=True)
loc_lat_txt = Text(value='', placeholder='',description='Lat. [°N]',disabled=True)
loc_box = VBox([loc_header_txt, loc_number_txt, loc_lon_txt, loc_lat_txt])
loc_box.layout.display = 'none'

#--------------------------------------------------------------------------------------------------------------------------------------
# FOOTER
#--------------------------------------------------------------------------------------------------------------------------------------

#-------------------------
## figures
#-------------------------

fig_zeta = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_zeta.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_velo = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_velo.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_time = go.FigureWidget(go.Scatter(x=[], y=[]))
fig_time.update_layout(autosize=True, margin=dict(l=10, r=10, t=30, b=30))
fig_zeta_box = VBox([fig_zeta])
fig_velo_box = VBox([fig_velo])
fig_time_box = VBox([fig_time])
fig_zeta_box.layout.display = 'none'
fig_velo_box.layout.display = 'none'
fig_time_box.layout.display = 'none'

# pdf/cdf helper
def pdf_cdf(samples, bins=40):
    hist, edges = np.histogram(samples, bins=bins, density=True)
    centers = (edges[:-1] + edges[1:]) / 2
    sorted_s = np.sort(samples)
    cdf = np.arange(1, len(sorted_s)+1)/len(sorted_s)
    return centers, hist, sorted_s, cdf
# create a single FigureWidget
def create_plot_widget(fig, arr, option, title, color="blue"):
    x_pdf, y_pdf, x_cdf, y_cdf = pdf_cdf(arr)
    fig.data = []  # clear old traces
    if option == 1:
        base = datetime.datetime(1970, 1, 1)
        x_pdf = [base + datetime.timedelta(minutes=float(m)) for m in x_pdf]
        x_cdf = [base + datetime.timedelta(minutes=float(m)) for m in x_cdf]
    fig.add_bar(x=x_pdf, y=y_pdf, marker_color=color, opacity=0.5, name="PDF")
    fig.add_scatter(x=x_cdf, y=y_cdf, mode='lines', line=dict(color="red", width=2),
                    name="CDF", yaxis="y2")
    fig.update_layout(
        title=title,
        yaxis=dict(title="PDF"),
        yaxis2=dict(title="CDF", overlaying="y", side="right"),
        template="simple_white",
    )
    if option == 1:
        fig.update_xaxes(type="date", tickformat="%M:%S")

#-------------------------
## statistics
#-------------------------

stat_header_txt = HTML("<center><h3>Probability [%]</h3></center>", layout=Layout(height='auto',width='99%'))

stat_ele_header_txt =  HTML("<h4>Elevation [m]</h4>", layout=Layout(height='auto',width='auto'))
stat_ele1_txt = Text(value='', placeholder='',description='P(<0.5)',disabled=True)
stat_ele2_txt = Text(value='', placeholder='',description='P(0.5-1.0)',disabled=True)
stat_ele3_txt = Text(value='', placeholder='',description='P(1.0-3.0)',disabled=True)
stat_ele4_txt = Text(value='', placeholder='',description='P(3.0-5.0)',disabled=True)
stat_ele5_txt = Text(value='', placeholder='',description='P(5.0-10.0)',disabled=True)
stat_ele6_txt = Text(value='', placeholder='',description='P(>10.0)',disabled=True)
stat_ele_box = VBox([stat_ele_header_txt, stat_ele1_txt, stat_ele2_txt, stat_ele3_txt, stat_ele4_txt, stat_ele5_txt, stat_ele6_txt])

stat_time_header_txt =  HTML("<h4>Time of Arrival [min]</h4>", layout=Layout(height='auto',width='auto'))
stat_time1_txt = Text(value='', placeholder='',description='P(<1)',disabled=True)
stat_time2_txt = Text(value='', placeholder='',description='P(1-2)',disabled=True)
stat_time3_txt = Text(value='', placeholder='',description='P(2-3)',disabled=True)
stat_time4_txt = Text(value='', placeholder='',description='P(3-5)',disabled=True)
stat_time5_txt = Text(value='', placeholder='',description='P(5-10)',disabled=True)
stat_time6_txt = Text(value='', placeholder='',description='P(>10)',disabled=True)
stat_time_box = VBox([stat_time_header_txt, stat_time1_txt, stat_time2_txt, stat_time3_txt, stat_time4_txt, stat_time5_txt, stat_time6_txt])

stats_box = VBox([stat_header_txt, HBox([stat_ele_box, stat_time_box])])
stats_box.layout.display = 'none'

#--------------------------------------------------------------------------------------------------------------------------------------
# FINAL LAYOUT
#--------------------------------------------------------------------------------------------------------------------------------------

AppLayout(header=header,
          left_sidebar=VBox([instruction_txt,m],layout=Layout(display='flex',justify_content='center')), 
          center=VBox([CLEAR_selector, CLEAR_selector_out,
                       Label("Zone:"), zone_selector, zone_out,
                       Label("Area:"), area_selector, area_out,
                       param_box,
                       OK_box],
                      layout=Layout(display='flex',justify_content='center')),
          right_sidebar=VBox([img_widget,loc_box],layout=Layout(display='flex', align_items='center')),
          footer=HBox([VBox([fig_zeta_box, fig_velo_box]), VBox([fig_time_box, stats_box])],layout=Layout(display='flex', justify_content="center")),
          pane_widths=[3, 3, 3],
          pane_heights=[1, 6, 10]
         )

AppLayout(children=(HTML(value='<center><h1>Landslide-Tsurrogate Model v1.0 - Mayotte (France)</h1></center>',…